# Reservas de Hotel
*Elaborado por: Manuel Hernandez | Maria Alejandra Jaime | Matias | Viridiana Valencia*


In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import pandas_profiling
#%pip install pycountry
import pycountry as pc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('hotel_booking.csv')

# Data Wrangling

En primer lugar, se debe manipular la data para modificar o eliminar registros, bien sea porque no son relevantes al estudio, necesiten algún tipo de edición para su uso dentro del modelo predictivo, o presenten valores desconocidos que requieran ser imputados o eliminados, entre otros. Para ello, empezaremos por conocer los tipos de datos y las cantidades de registros existentes de cada feature del dataset.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

# Eliminar: agent, company, name, email, phone, credit card

In [4]:
prelim_df = df.copy()
prelim_df = prelim_df.drop(columns=["agent", "company", "name", "email", "phone-number", "credit_card"], axis=1)
prelim_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,C,3,No Deposit,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,C,4,No Deposit,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,C,0,No Deposit,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,A,0,No Deposit,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,A,0,No Deposit,0,Transient,98.0,0,1,Check-Out,2015-07-03


# Valores Missing

In [5]:
features = prelim_df.columns.to_series()
nan_counts = prelim_df.count()
df_nan = pd.DataFrame(nan_counts).set_index(features)
filt = df_nan[0] != df.shape[0]
df_nan[filt]

,0
children,119386
country,118902


In [6]:
# Children
prelim_df.children.fillna(0,inplace=True)
df_nan = pd.DataFrame(prelim_df.count()).set_index(features)
df_nan[filt]

,0
children,119390
country,118902


In [7]:
# Country
prelim_df.country.fillna('Unavailable',inplace=True)
df_nan = pd.DataFrame(prelim_df.count()).set_index(features)
df_nan[filt]

,0
children,119390
country,119390


# Eliminar registros donde no hayan huéspedes adultos.

In [8]:
# Cambiamos el tipo de variable de children a int
prelim_df["children"] = prelim_df["children"].astype(int)

In [9]:
# Revisa si adultos == 0 y si es asi cambia adultos = 1 y niños = 0
no_adult_df = prelim_df[(prelim_df["adults"] == 0) & ((prelim_df["children"] != 0)|(prelim_df["babies"] != 0) )]["adults"]
print("Registros que no tienen huespedes adultos pero huespedes de otro tipo -> ", no_adult_df.count())
print("Dimension del dataframe actual -> ", prelim_df.shape)

Registros que no tienen huespedes adultos pero huespedes de otro tipo ->  223
Dimension del dataframe actual ->  (119390, 30)


In [10]:
## Elimina los registros que cumplen la condicion anterior
prelim_df = prelim_df.drop(no_adult_df.index)
print("Dimension del dataframe actual -> ", prelim_df.shape)

Dimension del dataframe actual ->  (119167, 30)


# Nuevas Features
## Feature total huespedes (total_guests)

In [11]:
prelim_df["total_guests"] = prelim_df["adults"] + prelim_df["children"] + prelim_df["babies"]
prelim_df.head().T

,0,1,2,3,4
hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel
is_canceled,0,0,0,0,0
lead_time,342,737,7,13,14
arrival_date_year,2015,2015,2015,2015,2015
arrival_date_month,July,July,July,July,July
arrival_date_week_number,27,27,27,27,27
arrival_date_day_of_month,1,1,1,1,1
stays_in_weekend_nights,0,0,0,0,0
stays_in_week_nights,0,0,1,1,2
adults,2,2,1,1,2


## Feature total días en el hotel (total_stay_in_nights)

In [12]:
prelim_df["total_stay_in_nights"] = prelim_df["stays_in_week_nights"] + prelim_df["stays_in_weekend_nights"]
prelim_df.head().T

,0,1,2,3,4
hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel
is_canceled,0,0,0,0,0
lead_time,342,737,7,13,14
arrival_date_year,2015,2015,2015,2015,2015
arrival_date_month,July,July,July,July,July
arrival_date_week_number,27,27,27,27,27
arrival_date_day_of_month,1,1,1,1,1
stays_in_weekend_nights,0,0,0,0,0
stays_in_week_nights,0,0,1,1,2
adults,2,2,1,1,2


In [13]:
# Cantidad de registros que tienen noches = 0 y que no fueron cancelados --- Son todos Check-Out
nfilt = (prelim_df["total_stay_in_nights"] == 0) & (prelim_df["is_canceled"] != 1)
prelim_df[nfilt]["reservation_status"].value_counts()

Check-Out    680
Name: reservation_status, dtype: int64

## Feature que indique si hay niños en la reserva (has_minors)

In [14]:
prelim_df["total_minors"] = prelim_df["children"] + prelim_df["babies"]
minors_filt = prelim_df["total_minors"] > 0
prelim_df[minors_filt]["total_minors"].value_counts()

1     5442
2     3567
3       97
10       2
9        1
Name: total_minors, dtype: int64

In [15]:
def hasMinors(minors):
    val = 0
    if (minors > 0):
        val = 1
    return val

prelim_df["total_minors"] = prelim_df["total_minors"].apply(hasMinors)
prelim_df = prelim_df.rename(columns={"total_minors": "has_minors"})
prelim_df.head(15).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel
is_canceled,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0
lead_time,342,737,7,13,14,14,0,9,85,75,23,35,68,18,37
arrival_date_year,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015
arrival_date_month,July,July,July,July,July,July,July,July,July,July,July,July,July,July,July
arrival_date_week_number,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27
arrival_date_day_of_month,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
stays_in_weekend_nights,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
stays_in_week_nights,0,0,1,1,2,2,2,2,3,3,4,4,4,4,4
adults,2,2,1,1,2,2,2,2,2,2,2,2,2,2,2


# Feature binario que compare el tipo de cuarto asignado con el tipo de cuarto reservado e indique si este cambió (room_type_changed)

In [16]:
# Asumo que para todos los registros assigned_room_type y reserved_room_type son iguales, asignandole cero inicialmente
prelim_df["room_type_changed"] = 0 
room_filt = (prelim_df["assigned_room_type"] != prelim_df["reserved_room_type"])
prelim_df.loc[room_filt, "room_type_changed"] = 1
prelim_df["room_type_changed"].value_counts()

0    104282
1     14885
Name: room_type_changed, dtype: int64

## Feature binario que indique si el cliente ha tenido más cancelaciones que estadías (cancelations_ratio)

In [17]:
#prelim_df['cancelations_ratio'] = 0
#prelim_df.loc[ prelim_df['previous_cancellations'] > ['previous_bookings_not_canceled'], 'cancelations_ratio'] = prelim_df['previous_cancellations'] / prelim_df['previous_bookings_not_canceled']

In [18]:
prelim_df['cancelations_ratio'] = 0
prelim_df['total_reservations'] = prelim_df['previous_cancellations'] + prelim_df['previous_bookings_not_canceled']
prelim_df.loc[prelim_df['total_reservations'] >0, 'cancelations_ratio'] = prelim_df['previous_cancellations'] / (prelim_df['previous_bookings_not_canceled'] + prelim_df['previous_cancellations'])

In [19]:
prelim_df.sort_values('total_reservations', ascending=False )

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,total_guests,total_stay_in_nights,has_minors,room_type_changed,cancelations_ratio,total_reservations
76731,City Hotel,0,0,2017,August,34,22,0,1,1,...,0,2,Check-Out,2017-08-23,1,1,0,1,0.076923,78
76730,City Hotel,0,4,2017,July,30,23,2,1,1,...,0,2,Check-Out,2017-07-26,1,3,0,0,0.077922,77
76729,City Hotel,0,4,2017,July,29,16,1,0,1,...,0,1,Check-Out,2017-07-17,1,1,0,0,0.078947,76
76728,City Hotel,0,5,2017,July,28,11,0,2,1,...,0,2,Check-Out,2017-07-13,1,2,0,0,0.080000,75
76727,City Hotel,0,5,2017,July,27,4,0,3,1,...,0,1,Check-Out,2017-07-07,1,3,0,0,0.081081,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41664,City Hotel,0,2,2015,August,34,21,0,2,1,...,0,0,Check-Out,2015-08-23,1,2,0,0,0.000000,0
41663,City Hotel,0,2,2015,August,34,21,0,2,2,...,0,2,Check-Out,2015-08-23,2,2,0,0,0.000000,0
41662,City Hotel,0,10,2015,August,34,21,0,1,2,...,0,2,Check-Out,2015-08-22,2,1,0,0,0.000000,0
41661,City Hotel,0,22,2015,August,34,21,0,1,2,...,0,0,Check-Out,2015-08-22,2,1,0,0,0.000000,0


# Eliminaciones

## Registros donde la cantidad de Adultos es igual a cero

In [20]:
prelim_df[(prelim_df["adults"] == 0)]["adults"].count()

180

In [21]:
adults_filt = (prelim_df["adults"] == 0)
prelim_df = prelim_df.drop(prelim_df[adults_filt].index)
prelim_df[(prelim_df["adults"] == 0)]["adults"].count()

0

## Registros con adr negativo o adr mayor a 400

In [22]:
adr_filt = (prelim_df["adr"] < 0) | (prelim_df["adr"] > 300)
len(prelim_df[adr_filt])

277

In [23]:
print("Valor máximo de ADR en el dataset --> ", prelim_df["adr"].max())
print("Valor mínimo de ADR en el dataset --> ", prelim_df["adr"].min())

Valor máximo de ADR en el dataset -->  5400.0
Valor mínimo de ADR en el dataset -->  -6.38


In [24]:
adr_filt = (prelim_df["adr"] < 0) | (prelim_df["adr"] > 300)
prelim_df = prelim_df.drop(prelim_df[adr_filt].index)
print("Valor máximo de ADR en el dataset --> ", prelim_df["adr"].max())
print("Valor mínimo de ADR en el dataset --> ", prelim_df["adr"].min())

Valor máximo de ADR en el dataset -->  300.0
Valor mínimo de ADR en el dataset -->  0.0


## Registros con tipos de habitacion P y L

In [25]:
assigned_filt = (prelim_df["assigned_room_type"] == 'P') | (prelim_df["assigned_room_type"] == 'L')
reserved_filt = (prelim_df["reserved_room_type"] == 'P') | (prelim_df["reserved_room_type"] == 'L')
prelim_df[assigned_filt | reserved_filt][["assigned_room_type", "reserved_room_type"]].value_counts()

assigned_room_type  reserved_room_type
A                   L                     1
B                   L                     1
C                   L                     1
F                   L                     1
H                   L                     1
L                   L                     1
dtype: int64

In [26]:
prelim_df = prelim_df.drop(prelim_df[assigned_filt | reserved_filt].index)
len(prelim_df[assigned_filt | reserved_filt][["assigned_room_type", "reserved_room_type"]])

0

# Modificación de tipo de datos 
## Features is_canceled, is_repeated_guest, cancelations_ratio, room_type_changed y has_minors se convierten a tipo object

In [27]:
prelim_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118704 entries, 0 to 119389
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           118704 non-null  object 
 1   is_canceled                     118704 non-null  int64  
 2   lead_time                       118704 non-null  int64  
 3   arrival_date_year               118704 non-null  int64  
 4   arrival_date_month              118704 non-null  object 
 5   arrival_date_week_number        118704 non-null  int64  
 6   arrival_date_day_of_month       118704 non-null  int64  
 7   stays_in_weekend_nights         118704 non-null  int64  
 8   stays_in_week_nights            118704 non-null  int64  
 9   adults                          118704 non-null  int64  
 10  children                        118704 non-null  int32  
 11  babies                          118704 non-null  int64  
 12  meal            

In [28]:
#prelim_df[['is_canceled', 'is_repeated_guest', 'room_type_changed', 'has_minors']] = prelim_df[['is_canceled', 'is_repeated_guest', 'room_type_changed', 'has_minors']].astype('object')
#prelim_df.info()

# Reinicio de los indices post Data Wrangling

In [29]:
data = prelim_df.copy()
data.reset_index()
print('Dimensiones iniciales --> ', df.shape)
print('Dimensiones luego de Data Wrangling --> ', data.shape)
print('Registros Eliminados --> ', df.shape[0]- data.shape[0])
print('Registros Eliminados (%) --> ', (df.shape[0]- data.shape[0])*100/df.shape[0])

Dimensiones iniciales -->  (119390, 36)
Dimensiones luego de Data Wrangling -->  (118704, 36)
Registros Eliminados -->  686
Registros Eliminados (%) -->  0.5745874863891448


# Algoritmos de Decisión

### En este ejercicio utilizaremos un modelo de regresión logística.

Empezamos por analizar las correlaciones que existen con la variable 'is_canceled'.

In [30]:
cancel_corr = data.corr()["is_canceled"]
cancel_corr.abs().sort_values(ascending=False)[1:]

lead_time                         0.293168
cancelations_ratio                0.292330
room_type_changed                 0.246907
total_of_special_requests         0.234847
required_car_parking_spaces       0.195291
booking_changes                   0.144679
previous_cancellations            0.110305
is_repeated_guest                 0.083603
adults                            0.059291
previous_bookings_not_canceled    0.057453
days_in_waiting_list              0.054509
adr                               0.050348
total_guests                      0.045207
babies                            0.032366
stays_in_week_nights              0.025332
total_stay_in_nights              0.018269
arrival_date_year                 0.016298
has_minors                        0.013891
arrival_date_week_number          0.008717
arrival_date_day_of_month         0.005686
children                          0.004437
total_reservations                0.003899
stays_in_weekend_nights           0.001651
Name: is_ca

### Seleccionamos los datos que utitilizaremos para el modelo.

In [31]:
data_seleccionada = data[['is_canceled','deposit_type','hotel','lead_time','cancelations_ratio','room_type_changed','total_of_special_requests','required_car_parking_spaces','booking_changes','is_repeated_guest','days_in_waiting_list','adr','total_stay_in_nights']]

In [32]:
data_seleccionada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118704 entries, 0 to 119389
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   is_canceled                  118704 non-null  int64  
 1   deposit_type                 118704 non-null  object 
 2   hotel                        118704 non-null  object 
 3   lead_time                    118704 non-null  int64  
 4   cancelations_ratio           118704 non-null  float64
 5   room_type_changed            118704 non-null  int64  
 6   total_of_special_requests    118704 non-null  int64  
 7   required_car_parking_spaces  118704 non-null  int64  
 8   booking_changes              118704 non-null  int64  
 9   is_repeated_guest            118704 non-null  int64  
 10  days_in_waiting_list         118704 non-null  int64  
 11  adr                          118704 non-null  float64
 12  total_stay_in_nights         118704 non-null  int64  
dtyp

### Convertimos en dummies la variables categóricas

In [33]:
data_seleccionada = pd.get_dummies(data_seleccionada, columns=['deposit_type'])
data_seleccionada = pd.get_dummies(data_seleccionada, columns=['hotel'],drop_first=True)
data_seleccionada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118704 entries, 0 to 119389
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   is_canceled                  118704 non-null  int64  
 1   lead_time                    118704 non-null  int64  
 2   cancelations_ratio           118704 non-null  float64
 3   room_type_changed            118704 non-null  int64  
 4   total_of_special_requests    118704 non-null  int64  
 5   required_car_parking_spaces  118704 non-null  int64  
 6   booking_changes              118704 non-null  int64  
 7   is_repeated_guest            118704 non-null  int64  
 8   days_in_waiting_list         118704 non-null  int64  
 9   adr                          118704 non-null  float64
 10  total_stay_in_nights         118704 non-null  int64  
 11  deposit_type_No Deposit      118704 non-null  uint8  
 12  deposit_type_Non Refund      118704 non-null  uint8  
 13 

### Preparamos los datos

In [34]:
Y = data_seleccionada["is_canceled"]
X = data_seleccionada.drop("is_canceled", axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=43)

### Creamos una instancia de regresión logística.

In [35]:
regresion_logistica = LogisticRegression()

### Entrenamos el modelo

In [36]:
regresion_logistica.fit(X,Y)

LogisticRegression()

In [37]:
prediccion = regresion_logistica.predict(X_test)

### Imprimimos el resultado

In [38]:
score = accuracy_score(Y_test, prediccion)
conf = confusion_matrix(Y_test, prediccion)
report = classification_report(Y_test, prediccion)

print(f"Puntaje de precisión: {score}\n")
print(f"Matriz de confusión:\n \n{conf}\n")
print(f"Reporte de clasificación: \n\n{report}")

Puntaje de precisión: 0.7759743906548354

Matriz de confusión:
 
[[21332  1097]
 [ 6881  6302]]

Reporte de clasificación: 

              precision    recall  f1-score   support

           0       0.76      0.95      0.84     22429
           1       0.85      0.48      0.61     13183

    accuracy                           0.78     35612
   macro avg       0.80      0.71      0.73     35612
weighted avg       0.79      0.78      0.76     35612



### El modelo de regresión logística obtuvo 77.6% de presición.

### Se clasificaron 22,429 observaciones como no cancelados, y de estas observaciones 21.331 se clasificaron de manera correcta. El 95% de las observaciones no canceladas se clasificó correctamente, mientras que 6.302 observaciones clasificadas como canceladas fueron predichas de manera correcta. Un total de 7.978 registros se clasificaron de manera incorrecta. 

# Random Forest

In [39]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
rand_forest = RandomForestClassifier(random_state=43, n_estimators=200)
rand_forest.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=200, random_state=43)

In [41]:
prediccion_rf = rand_forest.predict(X_test) 

In [42]:
score_rf = accuracy_score(Y_test, prediccion_rf)
conf_rf = confusion_matrix(Y_test, prediccion_rf)
report_rf = classification_report(Y_test, prediccion_rf)

print(f"Puntaje de precisión: {score_rf}\n")
print(f"Matriz de confusión:\n \n{conf_rf}\n")
print(f"Reporte de clasificación: \n\n{report_rf}")

Puntaje de precisión: 0.834522071211951

Matriz de confusión:
 
[[20154  2275]
 [ 3618  9565]]

Reporte de clasificación: 

              precision    recall  f1-score   support

           0       0.85      0.90      0.87     22429
           1       0.81      0.73      0.76     13183

    accuracy                           0.83     35612
   macro avg       0.83      0.81      0.82     35612
weighted avg       0.83      0.83      0.83     35612



### El modelo random forest obtuvo 83.45% de precisión.

### Se clasificaron 22.429 observaciones como no cancelados, y de estas observaciones 20.154 se clasificaron de manera correcta. El 90% de las observaciones no canceladas se clasificó correctamente, mientras que 9.565 observaciones clasificadas como canceladas fueron predichas de manera correcta. Un total de 5.893 registros se clasificaron de manera incorrecta. 

## Conclusión

### Al comparar ambos modelos, podemos ver que la regresión logística resulta en una precisión menor que el algoritmo de random forest. También podemos observar, a partir de la matriz de confusión de cada modelo, que el porcentaje de falsos positivos obtenido es menor en el modelo de regresión logística en comparación al random forest. Sin embargo, la regresión logística presenta una mayor cantidad de valores clasificados de manera incorrecta, bien sean falsos positivos o negativos, lo cual se ve reflejado en un valor f1 mayor para ambas clasificaciones en el modelo de random forest.